### This script contains the following points:

#### 1. Importing libraries
#### 2. Importing orders_products_merged
#### 3. Create price_range column and labels for product prices
#### 4. Create price_label and busiest_day columns
#### 5. Create new column for busiest_period_of_day
#### 6. Exporting ords_prods_merge to pkl

# 1. Importing libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

# 2. Importing orders_products_merged

In [2]:
# importing PKL order_products_combined from prepared data
ords_prods_merge = pd.read_pickle(r'C:\Users\kevan\Documents\Career Foundry\Data Immersion\Achievement 4\Instacart Basket Analysis\02 Data\Prepared Data\orders_products_merged.pkl')

In [3]:
# assigning main project path to variable 'path'
path = r'C:\Users\kevan\Documents\Career Foundry\Data Immersion\Achievement 4\Instacart Basket Analysis'

# 3. Create price_range column and labels for product prices

In [4]:
#subset of dataset to conserve memory
df = ords_prods_merge[:1000000]

In [5]:
df.shape

(1000000, 13)

In [7]:
# creating a function to assign labels to products based on price

def price_label(row):

    if row['prices'] <= 5:
        return 'Low-range product'
    elif (row['prices'] > 5) and (row['prices'] <= 15):
        return 'Mid-range product'
    elif row['prices'] > 15:
        return 'High range'
    else: return 'Not enough data'

In [8]:
#apply the function by filling values in new price_range column

df['price_range'] = df.apply(price_label, axis=1)

C:\Users\kevan\AppData\Local\Temp\ipykernel_8436\3743213296.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis=1)


In [9]:
#counts of values in different ranges for first 1 million rows

df['price_range'].value_counts(dropna = False)

Mid-range product    756450
Low-range product    243550
Name: price_range, dtype: int64

In [10]:
#finding the max price in subset, result confirms that there are no high range items in the subset

df['prices'].max()

14.8

# 4. Create price_label and busiest_day columns to ords_prods_merge

In [11]:
# checking shape for reference
ords_prods_merge.shape

(32404859, 13)

In [12]:
# similar to function in section 3, but faster
#filters are applied before searching data frame (rather than search then apply for if/then)

ords_prods_merge.loc[ords_prods_merge['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [13]:
ords_prods_merge.loc[(ords_prods_merge['prices'] <= 15) & (ords_prods_merge['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [14]:
ords_prods_merge.loc[ords_prods_merge['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [15]:
#able to search entire df after using loc() function

ords_prods_merge['price_range_loc'].value_counts(dropna = False)

Mid-range product     21860860
Low-range product     10126321
High-range product      417678
Name: price_range_loc, dtype: int64

### Counts of orders for each day of week. Saturday (0) is busiest and Wednesday (4) is slowest.

In [16]:
# number of orders by day of week
ords_prods_merge['orders_day_of_week'].value_counts(dropna = False)

0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
Name: orders_day_of_week, dtype: int64

### Using If-statements to create labels for the busiest days (0 & 1), slowest days (3 & 4) and regularly busy days (2,5,6)

In [17]:
# condition will create busiest_day column and apply labels to each day of week

result = []

for value in ords_prods_merge["orders_day_of_week"]:
    if value <= 1:
        result.append("Busiest")
    elif value == 4 and value == 3:
        result.append("Slowest")
    else:
        result.append("Regularly busy")
ords_prods_merge["busiest_day"] = result
ords_prods_merge

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32404854,1320836,202557,17,2,15,1.0,43553,2,1,Orange Energy Shots,64,7,3.7,Low-range product,Regularly busy
32404855,31526,202557,18,5,11,3.0,43553,2,1,Orange Energy Shots,64,7,3.7,Low-range product,Regularly busy
32404856,758936,203436,1,2,7,NaN,42338,4,0,"Zucchini Chips, Pesto",50,19,6.9,Mid-range product,Regularly busy
32404857,2745165,203436,2,3,5,15.0,42338,16,1,"Zucchini Chips, Pesto",50,19,6.9,Mid-range product,Regularly busy


In [18]:
#counts of each category which match the counts of DOW calculated before
ords_prods_merge['busiest_day'].value_counts(dropna = False)

Regularly busy    20540447
Busiest           11864412
Name: busiest_day, dtype: int64

In [19]:
# checking shape for reference
ords_prods_merge.shape

(32404859, 15)

In [20]:
#successful addition of new column and new results
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy


# 5.  Create new column for busiest_period_of_day

In [21]:
# busiest hours of day: most orders, average, fewest
# 24 hours of day divided by three time frame labels
#counts for orders per hour of day calculated and sorted by most orders

ords_prods_merge.value_counts(['order_hour_of_day'])

order_hour_of_day
10                   2761760
11                   2736140
14                   2689136
15                   2662144
13                   2660954
12                   2618532
16                   2535202
9                    2454203
17                   2087654
8                    1718118
18                   1636502
19                   1258305
20                    976156
7                     891054
21                    795637
22                    634225
23                    402316
6                     290493
0                     218769
1                     115700
5                      87961
2                      69375
4                      53242
3                      51281
dtype: int64

In [22]:
# hours greater than or equal to 9 are Most orders
# hours 7, 8, and 17-22 are average orders
# all other hours are fewest orders

result = []

for value in ords_prods_merge["order_hour_of_day"]:
    if value in [9, 10, 11, 12, 13, 14, 15, 16]:
        result.append("Most orders")
    elif value in [17, 8, 18, 19, 20, 7, 21, 22]:
        result.append("Average orders")
    else:
        result.append("Fewest orders")
ords_prods_merge["busiest_period_of_day"] = result
ords_prods_merge

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_period_of_day
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Average orders
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Average orders
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Most orders
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Average orders
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Most orders
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32404854,1320836,202557,17,2,15,1.0,43553,2,1,Orange Energy Shots,64,7,3.7,Low-range product,Regularly busy,Most orders
32404855,31526,202557,18,5,11,3.0,43553,2,1,Orange Energy Shots,64,7,3.7,Low-range product,Regularly busy,Most orders
32404856,758936,203436,1,2,7,NaN,42338,4,0,"Zucchini Chips, Pesto",50,19,6.9,Mid-range product,Regularly busy,Average orders
32404857,2745165,203436,2,3,5,15.0,42338,16,1,"Zucchini Chips, Pesto",50,19,6.9,Mid-range product,Regularly busy,Fewest orders


In [23]:
# confirm same number of rows. addition of one new column

ords_prods_merge.shape

(32404859, 16)

In [24]:
# confirm that all values are assigned labels

ords_prods_merge.value_counts(['busiest_period_of_day'])

busiest_period_of_day
Most orders              21118071
Average orders            9997651
Fewest orders             1289137
dtype: int64

# 6. Exporting ords_prods_merge to pkl

In [25]:
# Export data to pkl

ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged.pkl'))